In [ ]:
import time                                                                
import cv2
from stegano import lsb
from os.path import isfile,join
import time    
import cv2
import numpy as np
import math
import os
import shutil
from subprocess import call,STDOUT


# Function to split the message.
def split_string(split_str,count=10):
    per_c=math.ceil(len(split_str)/count)
    c_cout=0
    out_str=''
    split_list=[]
    for s in split_str:
        out_str+=s
        c_cout+=1
        if c_cout == per_c:
            split_list.append(out_str) # The message is divided into substrings
            out_str=''
            c_cout=0
    if c_cout!=0:
        split_list.append(out_str)
    return split_list


# As we know video is a collection of frames, where each frame is a picture.
# This function will extract each frame along with audio from the video.
def frame_extraction(video):
    if not os.path.exists("./temp"):
        os.makedirs("temp")
    temp_folder="./temp"            # Temporary folder created to store the frames and audio from the video.
    print("[INFO] temp directory is created")
    vidcap = cv2.VideoCapture(video)
    count = 0
    while True:
        success, image = vidcap.read()
        if not success:
            break
        cv2.imwrite(os.path.join(temp_folder, "{:d}.png".format(count)), image)
        count += 1


# This function would embed the splitted string into the frames extracted from the video.
def encode_string(input_string,root="./temp/"):
    split_string_list=split_string(input_string)   # Acquire the splitted string from the message.
    for i in range(0,len(split_string_list)):
        f_name="{}{}.png".format(root,i)                   
        secret_enc=lsb.hide(f_name,split_string_list[i])   # Embedded the splitted string into each frame.
        secret_enc.save(f_name)                            # Saved the frames after hidding the strings.
        print("[INFO] frame {} holds {}".format(f_name,lsb.reveal(f_name)))
    print("The message is stored in the Embedded_Video.mp4 file")


# This function would decode the hidden message by extracting frames from the video
def decode_string(video):
    frame_extraction(video)        # Extracting each frame from the video
    secret=[]
    root="./temp/"
    for i in range(len(os.listdir(root))):
        f_name="{}{}.png".format(root,i)
        secret_dec=lsb.reveal(f_name)         # Revealing the message inside each string
        if secret_dec == None:
            break
        secret.append(secret_dec)
    print(''.join([i for i in secret]))


# This function would extraxt audio from the video so as to stitch them back later.
def input_main(f_name):
    input_string = input("Enter the message :")   # To collect the message
    frame_extraction(f_name)
    # The call function would be used to extract the audio and then stitch it again properly with the frames extracted.
    call(["ffmpeg", "-i",f_name, "-q:a", "0", "-map", "a", "temp/audio.mp3", "-y"],stdout=open(os.devnull, "w"), shell=True, stderr=STDOUT)
    #call(["ffmpeg", "-i",f_name, "-q:a", "0", "-map", "a", "temp/audio.mp3", "-y"],stdout=open(os.devnull, "w"), stderr=STDOUT)
    encode_string(input_string)
    call(["ffmpeg", "-i", "temp/%d.png" , "-vcodec", "png", "temp/Embedded_Video.mp4", "-y"],stdout=open(os.devnull, "w"), shell=True, stderr=STDOUT)
    call(["ffmpeg", "-i", "temp/Embedded_Video.mp4", "-i", "temp/audio.mp3", "-codec", "copy", "Embedded_Video.mp4", "-y"],stdout=open(os.devnull, "w"), shell=True, stderr=STDOUT)


if __name__ == "__main__":    
    while True:
        print("1.Hide a message in video\n2.Reveal the secret from the video\n")
        print("Any other value to exit\n")
        choice = input("Enter your choice :")
        if choice == '1':
            f_name=input("Enter the name of video file with extension:")
            input_main(f_name)
        elif choice == '2':
            decode_string(input("Enter the name of video with extension :"))
        else:
            break

1.Hide a message in video
2.Reveal the secret from the video

Any other value to exit

Enter your choice :1
Enter the name of video file with extension:vi.mp4
Enter the message :20BCE0439
[INFO] temp directory is created
[INFO] frame ./temp/0.png holds 2
[INFO] frame ./temp/1.png holds 0
[INFO] frame ./temp/2.png holds B
[INFO] frame ./temp/3.png holds C
[INFO] frame ./temp/4.png holds E
[INFO] frame ./temp/5.png holds 0
[INFO] frame ./temp/6.png holds 4
[INFO] frame ./temp/7.png holds 3
[INFO] frame ./temp/8.png holds 9
The message is stored in the Embedded_Video.mp4 file
1.Hide a message in video
2.Reveal the secret from the video

Any other value to exit

Enter your choice :2
Enter the name of video with extension :Embedded_video.mp4
[INFO] temp directory is created
20BCE0439
1.Hide a message in video
2.Reveal the secret from the video

Any other value to exit

